In [ ]:
!pip install pyarrow==14.0.2

In [ ]:
!pip install requests==2.31.0

In [ ]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install datasets==2.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0


In [ ]:
!pip install datasets

In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd
import json
from datasets import Dataset

# Load the dataset from the JSON file
with open('/content/alpaca-sinhala.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to a DataFrame
df = pd.DataFrame(data)

# Combine "instruction" and "input" for the prompt
df['prompt'] = df['instruction'] + " " + df['input']
df = df[['prompt', 'output']]

# Create a Hugging Face dataset
dataset = Dataset.from_pandas(df)


In [ ]:
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
special_tokens_dict = {'pad_token': '[PAD]'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
print(f"Added {num_added_tokens} special tokens.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Added 1 special tokens.


In [ ]:
def tokenize_function(examples):
    inputs = tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=64)
    targets = tokenizer(examples["output"], truncation=True, padding="max_length", max_length=64)

    inputs["input_ids"] = [input_id + target_id for input_id, target_id in zip(inputs["input_ids"], targets["input_ids"])]
    inputs["attention_mask"] = [input_mask + target_mask for input_mask, target_mask in zip(inputs["attention_mask"], targets["attention_mask"])]
    inputs["labels"] = inputs["input_ids"].copy()

    max_token_id = max(max(input_id) for input_id in inputs["input_ids"])
    if max_token_id >= len(tokenizer):
        raise ValueError(f"Token ID {max_token_id} is out of range for tokenizer with vocab size {len(tokenizer)}.")

    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/49741 [00:00<?, ? examples/s]

In [ ]:
import torch
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    attention_mask = torch.tensor([item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

train_loader = DataLoader(tokenized_datasets, batch_size=2, collate_fn=collate_fn)

from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

In [ ]:
from transformers import GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

model.train()
accumulation_steps = 4

for epoch in range(3):
    optimizer.zero_grad()
    for step, batch in enumerate(train_loader):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with autocast():
            outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()

        if (step + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()
            optimizer.zero_grad()

        print(f"Epoch {epoch + 1}, Step {step + 1}, Loss: {loss.item()}")

# Save the model
save_directory = "./sinhala_fine_tuned_distilgpt2_2"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Epoch 1, Step 1, Loss: 28.631305694580078
Epoch 1, Step 2, Loss: 33.99980163574219
Epoch 1, Step 3, Loss: 24.668676376342773
Epoch 1, Step 4, Loss: 40.620079040527344
Epoch 1, Step 5, Loss: 35.81111526489258


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Streaming output truncated to the last 5000 lines.
Epoch 3, Step 19872, Loss: 1.4083448648452759
Epoch 3, Step 19873, Loss: 1.324100136756897
Epoch 3, Step 19874, Loss: 1.0356146097183228
Epoch 3, Step 19875, Loss: 2.096290111541748
Epoch 3, Step 19876, Loss: 0.38733699917793274
Epoch 3, Step 19877, Loss: 1.909152626991272
Epoch 3, Step 19878, Loss: 2.4554953575134277
Epoch 3, Step 19879, Loss: 1.6421046257019043
Epoch 3, Step 19880, Loss: 2.6898889541625977
Epoch 3, Step 19881, Loss: 0.8987845182418823
Epoch 3, Step 19882, Loss: 2.0192883014678955
Epoch 3, Step 19883, Loss: 0.9462364912033081
Epoch 3, Step 19884, Loss: 0.891612708568573
Epoch 3, Step 19885, Loss: 2.1051816940307617
Epoch 3, Step 19886, Loss: 0.6224548816680908
Epoch 3, Step 19887, Loss: 1.7662112712860107
Epoch 3, Step 19888, Loss: 0.604706346988678
Epoch 3, Step 19889, Loss: 1.5219805240631104
Epoch 3, Step 19890, Loss: 1.6945208311080933
Epoch 3, Step 19891, Loss: 1.6366322040557861
Epoch 3, Step 19892, Loss: 1.4842

('./sinhala_fine_tuned_distilgpt2_2/tokenizer_config.json',
 './sinhala_fine_tuned_distilgpt2_2/special_tokens_map.json',
 './sinhala_fine_tuned_distilgpt2_2/sentencepiece.bpe.model',
 './sinhala_fine_tuned_distilgpt2_2/added_tokens.json')

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path in your Google Drive where you want to save the model
drive_path = '/content/drive/MyDrive/sinhala_fine_tuned_distilgpt2_2'

# Copy the directory to Google Drive
shutil.copytree('./sinhala_fine_tuned_distilgpt2_2', drive_path)

print(f"Model saved to {drive_path}")

Mounted at /content/drive
Model saved to /content/drive/MyDrive/sinhala_fine_tuned_distilgpt2_2


In [ ]:
from transformers import GPT2LMHeadModel, XLMRobertaTokenizer
import torch

# Load the fine-tuned model and tokenizer
model_path = "./sinhala_fine_tuned_distilgpt2_2"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Define the prompt
prompt = "අක්ෂර වින්‍යාසය සහ ව්‍යාකරණ වැරදි සඳහා මෙම වාක්‍යය ඇගයීමට ලක් කරන්න: ඔහු තම ආහාර වේල සකසා ආපනශාලාවෙන් පිටව ගියේය"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors='pt')

# Ensure that model and inputs are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate text
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs['input_ids'],
        max_length=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id
    )

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
cx

අක්ෂර වින් යාසය සහ ව් යාකරණ වැරදි සඳහා මෙම වාක් යය ඇගයීමට ලක් කරන්න: ඔහු තම ආහාර වේල සකසා ආපනශාලාවෙන් පිටව ගියේය
